In [198]:
import re
import ic_model 
import pandas as pd
import numpy as np
from pathlib import Path
from datasets import load_dataset
# import nltk
# nltk.download('punkt')
pd.set_option('display.max_colwidth', 1000)

In [ ]:
test_ds = load_dataset('craigslist_bargains', split= 'validation')
test_ds = pd.DataFrame(test_ds)
test_ds.rename(columns={'utterance':'bargain_convo','dialogue_acts':'intent'}, inplace=True)
test_ds = test_ds.loc[6,:]

In [200]:
test_ds

agent_info                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 {'Bottomline': ['None', 'None'], 'Role': ['buyer', 'seller'], 'Target': [38.0, 50.0]}
agent_turn                                                                                                                                                                                                                                                                                             

In [201]:
# intent_timeline = np.array([['buyers_intent','buyers_bid','bots_intent','bots_bid']])
# np.save('intent_timeline.npy', intent_timeline)

In [ ]:
def priceExtraction(value):
    #searching for price-value or price-range in user input text string
    price = list(map(int, re.findall('(\$\d+)(\-*\d*)', value)))
    if price:
        return price[0][0][1:], price[0][1][1:] if price[0][1]!='' else 0
    else:
        return 0

def saveIntentIntoTimeline(data):
    #saving current buyer's and bot's intent and bids in file intent_timeline.npy
    intent_timeline = np.load('intent_timeline.npy')
    new_timeline = np.append(intent_timeline, [data], axis=0)
    np.save('intent_timeline.npy', new_timeline)
    print('current intents saved in intent_timeline', new_timeline)

In [209]:
def Intentagree(buyers_bid):
    #once the deal is done, reseting previous intent of ongoing conversation
    intent_timeline = np.array([['buyers_intent','buyers_bid','bots_intent','bots_bid']])
    np.save('intent_timeline.npy', intent_timeline)
    return 'agree'

def Intentintro(buyers_bid):
    saveIntentIntoTimeline(['intro',buyers_bid,'intro',buyers_bid])
    return 'intro'

def Intentinitprice(buyers_bid):
    bots_bid = 0 #yet to perform price prediction
    saveIntentIntoTimeline(('init-price',buyers_bid),('intro',bots_bid))
    return 'init-price'

def Intentcounterprice(buyers_bid):
    bots_bid = 0 #yet to perform price prediction
    saveIntentIntoTimeline(('counter-price',buyers_bid),('counter-price',bots_bid))
    return 'counter-price'    

In [ ]:
def decisionEngine(text):
    #function call for intentClassification from file ic_model.py
    buyers_intent = ic_model.predict_intent(text)
    buyers_intent = 'counterprice' if buyers_intent == 'counter-price' else buyers_intent
    buyers_intent = 'initprice' if buyers_intent == 'init-price' else buyers_intent

    #fetching discret price-range or buyers-bid from the user-input text i.e (rangeMin,rangeMax) or (singleValue,0) 
    #e.g.(100,200), (100,0) 
    buyers_bid = priceExtraction(text)

    #calling distint functions according to the buyers_intent
    return eval("Intent" + str(buyers_intent) + "({})".format(buyers_bid))

In [ ]:
decisionEngine("hey, how you doing?")